In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Lambda

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path='mnist.npz')


In [3]:
num_samples = x_train.shape[0]
num_classes = 10

In [4]:
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_train = ds_train.map(lambda x, y: (tf.reshape(x, [-1])/255, tf.one_hot(y, num_classes)))
ds_train = ds_train.batch(256).prefetch(1)

ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
ds_test = ds_test.map(lambda x, y: (tf.reshape(x, [-1])/255, tf.one_hot(y, num_classes)))
ds_test = ds_test.batch(256).prefetch(1)



In [5]:
tf.keras.backend.clear_session()


In [6]:
@tf.function
def cube_this(x):
    return x**3

In [7]:
from layers import Highway

model = Sequential()
model.add(Dense(512, activation = 'relu', input_shape = (784, )))
model.add(Dropout(0.2))
model.add(Highway())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation = 'softmax'))
model.build()

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
highway (Highway)            (None, 512)               525312    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                32832     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 960,714
Trainable params: 960,714
Non-trainable params: 0
__________________________________________________

In [9]:
model.compile('RMSprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(ds_train, epochs = 10)

Epoch 1/10
235/235 [==============================] - 6s 24ms/step - loss: 0.3806 - accuracy: 0.8995
Epoch 2/10
235/235 [==============================] - 4s 18ms/step - loss: 0.1751 - accuracy: 0.9550
Epoch 3/10
235/235 [==============================] - 4s 19ms/step - loss: 0.1344 - accuracy: 0.9652
Epoch 4/10
235/235 [==============================] - 4s 18ms/step - loss: 0.1126 - accuracy: 0.9706
Epoch 5/10
235/235 [==============================] - 4s 18ms/step - loss: 0.0979 - accuracy: 0.9743
Epoch 6/10
235/235 [==============================] - 4s 19ms/step - loss: 0.0923 - accuracy: 0.9763
Epoch 7/10
235/235 [==============================] - 5s 19ms/step - loss: 0.0946 - accuracy: 0.9771
Epoch 8/10
235/235 [==============================] - 5s 20ms/step - loss: 0.0771 - accuracy: 0.9806
Epoch 9/10
235/235 [==============================] - 5s 20ms/step - loss: 0.0771 - accuracy: 0.9813
Epoch 10/10
136/235 [================>.............] - ETA: 2s - loss: 0.0718 - accuracy: 0